In [1]:
import random
import time
import numpy as np
import math 
import cv2
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import PPO 
from typing import Callable
import os
import carla

2024-05-07 12:54:58.768630: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-07 12:54:58.841942: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-07 12:54:59.175735: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/gp-coopperc/anaconda3/envs/Tensor/lib/python3.9/site-packages/cv2/../../lib64:/usr/local/cud

In [2]:
SECONDS_PER_EPISODE = 20

N_CHANNELS = 3
HEIGHT = 250
WIDTH = 250

FIXED_DELTA_SECONDS = 0.2

SHOW_PREVIEW = True

In [3]:
class CarEnv(gym.Env):
    SHOW_CAM = SHOW_PREVIEW
    STEER_AMT = 1.0
    im_width = WIDTH
    im_height = HEIGHT
    front_camera = None
    CAMERA_POS_Z = 1.3 
    CAMERA_POS_X = 1.4
    PREFERRED_SPEED = 60
    SPEED_THRESHOLD = 2
    
    def __init__(self):
        super(CarEnv, self).__init__()
        
        self.action_space = spaces.Discrete(9)

        self.observation_space = spaces.Box(low=0.0, high=255.0,
                                            shape=(HEIGHT, WIDTH, N_CHANNELS), dtype=np.uint8)
        
        self.client = carla.Client("localhost", 2000)
        self.client.set_timeout(4.0)
        self.client.load_world('Town01')
        self.world = self.client.get_world()

        self.settings = self.world.get_settings()
        #self.settings.no_rendering_mode = True
        #self.settings.synchronous_mode = False
        #self.settings.fixed_delta_seconds = FIXED_DELTA_SECONDS
        self.world.apply_settings(self.settings)
        self.blueprint_library = self.world.get_blueprint_library()
        self.model_3 = self.blueprint_library.filter('*mini*')[0]
        self.actor_list = []
        
    def maintain_speed(self,s):
        
        if s >= self.PREFERRED_SPEED:
            return 0
        elif s < self.PREFERRED_SPEED - self.SPEED_THRESHOLD:
            return 0.7 # think of it as % of "full gas"
        else:
            return 0.3 # tweak this if the car is way over or under preferred speed 

        
    def reset(self, seed=1, options=None):
        super().reset(seed=seed)
        self.cleanup()


                
        self.collision_hist = []
        self.actor_list = []
        self.spawn_points = self.world.get_map().get_spawn_points()
        #print(self.spawn_points[2])
        #print(random.choice(self.spawn_points))
        self.start_point = self.spawn_points[2]
        
        # Define point A and point B
        self.point_a = self.start_point
        self.point_b = carla.Location(x=107.175446, y=129.416809, z=0.003263)  # Destination point
        
    
        print("Trying to spawn vehicle...")
        self.vehicle = self.world.try_spawn_actor(self.model_3, self.start_point)
        if self.vehicle is None:
            print("Failed to spawn vehicle! Resetting environment...")
            self.cleanup()  # Clean up any existing actors
            return self.reset()  # Retry resetting the environment
        
        print("Vehicle spawned successfully!")
        self.actor_list.append(self.vehicle)
        self.initial_location = self.vehicle.get_location()

        self.sem_cam = self.blueprint_library.find('sensor.camera.semantic_segmentation')
        self.sem_cam.set_attribute("image_size_x", f"{self.im_width}")
        self.sem_cam.set_attribute("image_size_y", f"{self.im_height}")
        self.sem_cam.set_attribute("fov", f"90")


        camera_init_trans = carla.Transform(carla.Location(z=self.CAMERA_POS_Z,x=self.CAMERA_POS_X))
        self.sensor = self.world.spawn_actor(self.sem_cam, camera_init_trans, attach_to=self.vehicle)
        self.actor_list.append(self.sensor)
        self.sensor.listen(lambda data: self.process_img(data))

        self.vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=0.0))
        time.sleep(2) #not detect a collision when the car spawns/falls from sky.
        # showing camera at the spawn point
        if self.SHOW_CAM:
            cv2.namedWindow('Sem Camera',cv2.WINDOW_AUTOSIZE)
            cv2.imshow('Sem Camera', self.front_camera)
            cv2.waitKey(1)
        colsensor = self.blueprint_library.find("sensor.other.collision")
        self.colsensor = self.world.spawn_actor(colsensor, camera_init_trans, attach_to=self.vehicle)
        self.actor_list.append(self.colsensor)
        self.colsensor.listen(lambda event: self.collision_data(event))
     

        while self.front_camera is None:
            time.sleep(0.01)#Just in case car takes any longer, because we need to be certain the car is done falling from the sky on spawn.

        self.episode_start = time.time()
        
        self.step_counter = 0
        self.vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=0.0))
        
        observation = self.front_camera

        return observation.astype(np.uint8), {}
    
    def step(self, action):
        self.step_counter +=1
        steer = action
        # map steering actions
        if steer ==0:
            steer = - 0.9
        elif steer ==1:
            steer = -0.25
        elif steer ==2:
            steer = -0.1
        elif steer ==3:
            steer = -0.05
        elif steer ==4:
            steer = 0.0 
        elif steer ==5:
            steer = 0.05
        elif steer ==6:
            steer = 0.1
        elif steer ==7:
            steer = 0.25
        elif steer ==8:
            steer = 0.9


        v = self.vehicle.get_velocity()
        kmh = int(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2))

        estimated_throttle = self.maintain_speed(kmh)

        self.vehicle.apply_control(carla.VehicleControl(throttle=estimated_throttle, steer=steer, brake = 0.0))
        
        distance_to_destination = self.vehicle.get_location().distance(self.point_b)
        distance = self.point_a.location.distance(self.point_b)

        #distance_travelled = self.initial_location.distance(self.vehicle.get_location())

        # storing camera to return at the end in case the clean-up function destroys it
        cam = self.front_camera
        # showing image
        if self.SHOW_CAM:
            cv2.imshow('Sem Camera', cam)
            cv2.waitKey(1)

        # start defining reward from each step
        reward = 0
        done = False
        #punish for collision
        if len(self.collision_hist) != 0:
            done = True
            reward = reward - 50
            #self.cleanup()
            # Reset the environment immediately after collision
            observation = self.reset()[0]
        #else:
            # Update observation if available, else use the last observation
            #observation = self.front_camera if self.front_camera is not None else self.last_observation

            
            
        # Calculate reward based on distance traveled towards point B    
        if distance_to_destination < distance/4:  # threshold for reaching destination
            reward = reward + 30  # Positive reward for reaching destination
            done = True  # End episode when destination is reached
        elif distance_to_destination < distance/3:
            reward = reward + 20  
        elif distance_to_destination < distance/2:
            reward = reward + 10  
        elif distance_to_destination < distance:
            reward = reward + 5
        
        else:
            reward = reward - 10  # Negative reward for not reaching destination
            done = False

        # check for episode duration
        if self.episode_start + SECONDS_PER_EPISODE < time.time():
            done = True
            self.cleanup()
            #print(self.actor_list)
            
        observation = cam

        return observation.astype(np.uint8), reward, done, False, {}
        
    def cleanup(self):
        for actor in self.actor_list:
            if actor.is_alive:
                actor.destroy()
        #self.actor_list = []
        cv2.destroyAllWindows()

        
    def process_img(self, image):
        image.convert(carla.ColorConverter.CityScapesPalette)
        i = np.array(image.raw_data)
        i = i.reshape((self.im_height, self.im_width, 4))[:, :, :3] # this is to ignore the 4th Alpha channel - up to 3
        self.front_camera = i/255.0

    def collision_data(self, event):
        self.collision_hist.append(event)

In [4]:
models_dir = f"models/{int(time.time())}/"
logdir = f"logs/{int(time.time())}/"

if not os.path.exists(models_dir):
    os.makedirs(models_dir)

if not os.path.exists(logdir):
    os.makedirs(logdir)


env = CarEnv()

env.reset()

model = PPO('CnnPolicy', env, verbose=1,learning_rate=0.001,tensorboard_log=logdir)

TIMESTEPS = 500_000 # individual steps
iters = 0
while iters<4:  # how many training iterations you want
    iters += 1
    print('Iteration ', iters,' is to start...')
    try:
        model.learn(total_timesteps=TIMESTEPS, reset_num_timesteps=False, tb_log_name=f"PPO")
        print('Iteration', iters, 'has been trained')
        model.save(f"{models_dir}/{TIMESTEPS*iters}")
    except ValueError as e:
        print(f"Error during training iteration {iters}: {e}")
        continue

Trying to spawn vehicle...
Vehicle spawned successfully!
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
Iteration  1  is to start...
Trying to spawn vehicle...
Vehicle spawned successfully!
Logging to logs/1715045341/PPO_0
-----------------------------
| time/              |      |
|    fps             | 350  |
|    iterations      | 1    |
|    time_elapsed    | 5    |
|    total_timesteps | 2048 |
-----------------------------


/home/gp-coopperc/anaconda3/envs/Tensor/lib/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 2.05e+03      |
|    ep_rew_mean          | 1.02e+04      |
| time/                   |               |
|    fps                  | 201           |
|    iterations           | 2             |
|    time_elapsed         | 20            |
|    total_timesteps      | 4096          |
| train/                  |               |
|    approx_kl            | 0.00095100456 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -2.2          |
|    explained_variance   | 0             |
|    learning_rate        | 0.001         |
|    loss                 | 3.5e+03       |
|    n_updates            | 10            |
|    policy_gradient_loss | -0.000807     |
|    value_loss           | 6.95e+03      |
------

Trying to spawn vehicle...
Vehicle spawned successfully!
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 2.05e+03      |
|    ep_rew_mean          | 1.02e+04      |
| time/                   |               |
|    fps                  | 153           |
|    iterations           | 10            |
|    time_elapsed         | 133           |
|    total_timesteps      | 20480         |
| train/                  |               |
|    approx_kl            | 0.00081087154 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -2.19         |
|    explained_variance   | 1.19e-07      |
|    learning_rate        | 0.001         |
|    loss                 | 3.42e+03      |
|    n_updates            | 90            |
|    policy_gradient_loss | -0.00104      |
|    value_loss           | 6.88e+03      |
-------------------------------------------
Trying to spawn veh

Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 2.05e+03     |
|    ep_rew_mean          | 1.09e+04     |
| time/                   |              |
|    fps                  | 150          |
|    iterations           | 19           |
|    time_elapsed         | 258          |
|    total_timesteps      | 38912        |
| train/                  |              |
|    approx_kl            | 0.0011251861 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.15        |
|    explained_variance   | 1.19e-07     |
|    learning_rate        | 0.001        |
|    loss                 | 3.34e+03     |
|    n_updates            | 180          |
|    policy_gradient_loss | -0.00149     |
|    value_loss           | 6.8e+03      |
--------------------------

...

Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 2.11e+03     |
|    ep_rew_mean          | 1.05e+04     |
| time/                   |              |
|    fps                  | 145          |
|    iterations           | 213          |
|    time_elapsed         | 2994         |
|    total_timesteps      | 1941504      |
| train/                  |              |
|    approx_kl            | 0.0016940502 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.81        |
|    explained_variance   | 0            |
|    learning_rate        | 0.001        |
|    loss                 | 530          |
|    n_updates            | 9470         |
|    policy_gradient_loss | -0.00177     |
|    value_loss           | 1.5e+03      |
--------------------------

Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 2.11e+03      |
|    ep_rew_mean          | 1.05e+04      |
| time/                   |               |
|    fps                  | 145           |
|    iterations           | 221           |
|    time_elapsed         | 3108          |
|    total_timesteps      | 1957888       |
| train/                  |               |
|    approx_kl            | 0.00060323003 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.79         |
|    explained_variance   | 0             |
|    learning_rate        | 0.001         |
|    loss                 | 532           |
|    n_updates            | 9550          |
|    policy_gradient_loss | -0.000606     |
|    value_loss           | 1.13e+03      |
------

Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 2.11e+03     |
|    ep_rew_mean          | 1.05e+04     |
| time/                   |              |
|    fps                  | 145          |
|    iterations           | 229          |
|    time_elapsed         | 3221         |
|    total_timesteps      | 1974272      |
| train/                  |              |
|    approx_kl            | 0.0018281688 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.75        |
|    explained_variance   | -1.19e-07    |
|    learning_rate        | 0.001        |
|    loss                 | 519          |
|    n_updates            | 9630         |
|    policy_gradient_loss | -0.00136     |
|    value_loss           | 1.1e+03      |
--------------------------

Trying to spawn vehicle...
Vehicle spawned successfully!
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 2.11e+03     |
|    ep_rew_mean          | 1.05e+04     |
| time/                   |              |
|    fps                  | 145          |
|    iterations           | 237          |
|    time_elapsed         | 3335         |
|    total_timesteps      | 1990656      |
| train/                  |              |
|    approx_kl            | 0.0008245171 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.74        |
|    explained_variance   | 0            |
|    learning_rate        | 0.001        |
|    loss                 | 582          |
|    n_updates            | 9710         |
|    policy_gradient_loss | -0.00113     |
|    value_loss           | 1.44e+03     |
------------------------------------------
Trying to spawn vehicle...
Vehicle spawn

Iteration 4 has been trained


In [4]:
env = CarEnv()
env.reset()

models_dir = "/home/gp-coopperc/CARLA_TEST/models/1715045341/"
model_path = f"{models_dir}/2000000.zip"
model = PPO.load(model_path, env=env)

Trying to spawn vehicle...
Vehicle spawned successfully!
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [4]:
env = CarEnv()
env.reset()

models_dir = "/home/gp-coopperc/CARLA_TEST/models/1714958790/"
model_path = f"{models_dir}/1500000.zip"
model = PPO.load(model_path, env=env)

Trying to spawn vehicle...
Vehicle spawned successfully!
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [5]:
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor


env = Monitor(env) 
evaluate_policy(model, env, n_eval_episodes=10,deterministic=True, return_episode_rewards= True)

Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!
Trying to spawn vehicle...
Vehicle spawned successfully!


([20155.0,
  19900.0,
  19780.0,
  19545.0,
  19465.0,
  19760.0,
  19720.0,
  19605.0,
  19530.0,
  19475.0],
 [2982, 2970, 2936, 2914, 2911, 2964, 2922, 2920, 2924, 2940])